In [1]:
from ROOT import TFile
from root_numpy import root2array, root2rec, tree2array
import array
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.callbacks import EarlyStopping
from keras.engine.topology import Input
from keras.engine.training import Model
from keras import losses, optimizers
from keras import backend as K
import math

Welcome to JupyROOT 6.10/09


Using TensorFlow backend.


In [2]:
config = tf.ConfigProto(intra_op_parallelism_threads=10, inter_op_parallelism_threads=10, \
                        allow_soft_placement=True, device_count = {'CPU': 10})
session = tf.Session(config = config)
K.set_session(session)

2018-03-19 19:12:52.303106: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-03-19 19:12:52.303134: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-03-19 19:12:52.303145: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.


In [3]:
def get_size(filepath):
    print "skimming " + filepath
    f = TFile.Open(filepath)
    size = f.Get("ZZTree/candTree").GetEntries()
    f.Close()

    return size

In [4]:
# this simulates a single ROOT file that is actually distributed over multiple "physical" ROOT trees. From each file in the list,
# only the portion between start_fraction and end_fraction is considered
class FileCollection:
    def __init__(self, files, start_fraction, end_fraction):
        self.files = files
        self.start_fraction = start_fraction
        self.end_fraction = end_fraction

        # the number of entries in each file, and the local start- and endpositions
        self.lengths = [get_size(file_path) for file_path in files]
        self.minpos = [int(length * start_fraction) for length in self.lengths]
        self.maxpos = [int(length * end_fraction) for length in self.lengths]
        
        self.used_lengths = [end - beginning for (end, beginning) in zip(self.maxpos, self.minpos)]
                
        self.total_length = sum(self.lengths)
        self.used_length = sum(self.used_lengths)
        
        print "collection set up: " + str(len(files)) + " files, " + str(self.total_length) + " entries in total, " + str(self.used_length) + " of which will be used"
        
    def get_length(self):
        return self.used_length
    
    # returns some data from this file collection
    def get_data(self, branches, start_index, end_index):
        # now need to translate between a global index, and a filepath and its corresponding local index
        return 0
        
    def transform_index(self, global_index):
        if global_index >= self.get_length():
            raise IndexError("global index out of range")
        
        # first determine which file in the list is needed to read this index
        max_local_indices = np.array(self.used_lengths) - 1  # all works by 0-indexing
        cum_lengths = np.cumsum(self.used_lengths)
        
        needed_file = 0
        while global_index > cum_lengths[needed_file] - 1:
            needed_file += 1
            
        # then determine the corresponding local index within this file
        local_minpos = np.append(0, cum_lengths)
                
        local_index = global_index - local_minpos[needed_file]
        
        # up to now, all these indices are relative w.r.t. the used slice in each file. the beginning of these slices can be shifted w.r.t. the beginning of the file itself
        local_index += self.minpos[needed_file]
            
        return self.files[needed_file], local_index
    
    def transform_index_range(self, global_start_index, global_end_index):
        if global_start_index >= self.get_length() or global_end_index >= self.get_length():
            raise IndexError("global index out of range")
        if global_end_index < global_start_index:
            raise IndexError("end ought to come after beginning")
            
        local_coords = [self.transform_index(global_index) for global_index in range(global_start_index, global_end_index)]
        needed_files = set([local_coord[0] for local_coord in local_coords])
        
        retval = []
        # now look at each needed file in turn and determine the relevant index range in this local file
        for needed_file in needed_files:
            needed_local_indices = [local_coord[1] for local_coord in local_coords if local_coord[0] == needed_file ]
            needed_min_index = min(needed_local_indices)
            needed_max_index = max(needed_local_indices)
            
            retval.append([needed_file, needed_min_index, needed_max_index])
        
        return retval

In [5]:
def generate_training_data(H1_files, H0_files, branches, training_split = 0.5, chunks = 1000):
    H1_collection = FileCollection(H1_files, start_fraction = 0.0, end_fraction = training_split)
    H0_collection = FileCollection(H0_files, start_fraction = 0.0, end_fraction = training_split)
    return datagen(H1_collection, H0_collection, branches, chunks = chunks)

In [6]:
def generate_validation_data(H1_files, H0_files, branches, training_split = 0.5, chunks = 1000):
    H1_collection = FileCollection(H1_files, start_fraction = training_split, end_fraction = 1.0)
    H0_collection = FileCollection(H0_files, start_fraction = training_split, end_fraction = 1.0)
    return datagen(H1_collection, H0_collection, branches, chunks = chunks)

In [7]:
def read_data(collection, start, stop, branches):
    #print "requesting data in range (" + str(start) + ", " + str(stop) + ")"
    filetuple = collection.transform_index_range(start, stop)
        
    files = [entry[0] for entry in filetuple]
    start_indices = [entry[1] for entry in filetuple]
    stop_indices = [entry[2] for entry in filetuple]
    
    read_list = []
    for (cur_file, cur_start_index, cur_stop_index) in zip(files, start_indices, stop_indices):
        #print "reading from " + cur_file + ": (" + str(cur_start_index) + ", " + str(cur_stop_index) + ")" 
        read_list.append(pd.DataFrame(root2array(cur_file, treename = "ZZTree/candTree", branches = branches, start = cur_start_index, stop = cur_stop_index)))
        #print "read successful"
        
    return pd.concat(read_list)

In [8]:
def datagen(H1_collection, H0_collection, branches, chunks = 100):
    H1_curpos = 0
    H0_curpos = 0

    H1_maxpos = H1_collection.get_length()
    H0_maxpos = H0_collection.get_length()

    print "H1 contains " + str(H1_maxpos) + " entries"
    print "H0 contains " + str(H0_maxpos) + " entries"

    H1_chunksize = int(H1_maxpos / chunks)
    H0_chunksize = int(H0_maxpos / chunks)
    
    print "using the following chunk sizes: " + "(" + str(H1_chunksize) + " / " + str(H0_chunksize) + ")"
    
    # as a generator, deliver data forever
    while True:
               
        if H1_curpos + H1_chunksize > H1_maxpos:
            H1_curpos = 0
            
        if H0_curpos + H0_chunksize > H0_maxpos:
            H0_curpos = 0
        
        # prepare next training data chunk set by drawing events randomly from the two files
        H1_data = read_data(H1_collection, branches = branches, start = H1_curpos, stop = H1_curpos + H1_chunksize)

        H0_data = read_data(H0_collection, branches = branches, start = H0_curpos, stop = H0_curpos + H0_chunksize)
                        
        # update the starting position for the next chunk
        H1_curpos += H1_chunksize
        H0_curpos += H0_chunksize

        # add the truth information
        H1_data["target"] = 1.0
        H0_data["target"] = 0.0

        data_chunk = pd.concat([H1_data, H0_data])

        # return a randomized signal + background sample
        training_data = data_chunk.sample(frac = 1)
        input_data = training_data[branches].as_matrix()
        target_data = training_data["target"].as_matrix()
        
        yield input_data, target_data

In [9]:
inpath = "/data_CMS/cms/wind/CJLST_NTuples/"
filename = "/ZZ4lAnalysis.root"
H1_files = ["VBFH125"]
H0_files = ["ggH125"]

H1_paths = [inpath + H1_file + filename for H1_file in H1_files]
H0_paths = [inpath + H0_file + filename for H0_file in H0_files]

In [10]:
branches = ["PFMET", "nCleanedJetsPt30"]

In [11]:
in_layer = Input(shape = (2,))
x = Dense(128, activation = "relu")(in_layer)
x = Dense(128, activation = "relu")(x)
x = Dense(128, activation = "relu")(x)
x = Dense(16, activation = "relu")(x)
out_layer = Dense(1, activation = "tanh", name = "out_layer")(x)
model = Model(in_layer, out_layer, name = "testmodel")

In [12]:
sgd = optimizers.SGD(lr = 0.1)
model.compile(loss = "mean_squared_error", optimizer = sgd, metrics = ["accuracy"])

In [13]:
train_gen = generate_training_data(H1_paths, H0_paths, branches, chunks = 100)
val_gen = generate_validation_data(H1_paths, H0_paths, branches, chunks = 100)

skimming /data_CMS/cms/wind/CJLST_NTuples/VBFH125/ZZ4lAnalysis.root
collection set up: 1 files, 62320 entries in total, 31160 of which will be used
skimming /data_CMS/cms/wind/CJLST_NTuples/ggH125/ZZ4lAnalysis.root
collection set up: 1 files, 110483 entries in total, 55241 of which will be used
skimming /data_CMS/cms/wind/CJLST_NTuples/VBFH125/ZZ4lAnalysis.root
collection set up: 1 files, 62320 entries in total, 31160 of which will be used
skimming /data_CMS/cms/wind/CJLST_NTuples/ggH125/ZZ4lAnalysis.root
collection set up: 1 files, 110483 entries in total, 55242 of which will be used


In [14]:
early_stop = EarlyStopping(monitor = 'val_loss',
                          patience = 10,
                          verbose = 1,
                          mode = 'auto')

In [15]:
ret = model.fit_generator(train_gen, steps_per_epoch = 128, epochs = 50, verbose = 2, validation_data = val_gen, validation_steps = 10, callbacks = [early_stop])

H1 contains 31160 entriesEpoch 1/50
H0 contains 55241 entries

using the following chunk sizes: (311 / 552)
H1 contains 31160 entries
H0 contains 55242 entries
using the following chunk sizes: (311 / 552)
11s - loss: 0.3008 - acc: 0.6044 - val_loss: 0.2304 - val_acc: 0.6400
Epoch 2/50
11s - loss: 0.2304 - acc: 0.6400 - val_loss: 0.2304 - val_acc: 0.6400
Epoch 3/50
10s - loss: 0.2304 - acc: 0.6400 - val_loss: 0.2304 - val_acc: 0.6400
Epoch 4/50
10s - loss: 0.2304 - acc: 0.6400 - val_loss: 0.2304 - val_acc: 0.6400
Epoch 5/50


KeyboardInterrupt: 